In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

from utils.df_loader import load_adult_df, load_compas_df, load_german_df, load_diabetes_df, load_breast_cancer_df
from sklearn.model_selection import train_test_split
from utils.preprocessing import preprocess_df
from utils.models import train_three_models, evaluation_test, save_three_models, load_three_models
from utils.cf_proto import  generate_cf_proto_result, process_result

from utils.save import save_result_as_csv

### Disable TF2 and enable TF1 for alibi.
tf.get_logger().setLevel(40) 
tf.compat.v1.disable_v2_behavior()
tf.keras.backend.clear_session()
pd.options.mode.chained_assignment = None 

print('TF version: ', tf.__version__)
print('Eager execution enabled: ', tf.executing_eagerly()) # False


seed = 123
tf.random.set_seed(seed)
np.random.seed(seed)


TF version:  2.4.0-rc0
Eager execution enabled:  False


In [2]:
#### Select dataset ####

dataset_name = 'diabetes' # [adult, german, compas]

if dataset_name == 'adult':
    dataset_loading_fn = load_adult_df
elif dataset_name == 'german':
    dataset_loading_fn = load_german_df
elif dataset_name == 'compas':
    dataset_loading_fn = load_compas_df
elif dataset_name == 'diabetes':
    dataset_loading_fn = load_diabetes_df
elif dataset_name == 'breast_cancer':
    dataset_loading_fn = load_breast_cancer_df
else:
    raise Exception("Unsupported dataset")

In [3]:
#### Load datafram info.
df_info = preprocess_df(dataset_loading_fn)

In [4]:
### Seperate to train and test set.
train_df, test_df = train_test_split(df_info.dummy_df, train_size=.8, random_state=seed, shuffle=True)

In [5]:
### Input queries for generating counterfactuals
df_info.df.iloc[test_df[:20].index]

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
236,7,181,84,21,192,35.9,0.586,51,Y
395,2,127,58,24,275,27.7,1.600,25,N
36,11,138,76,0,0,33.2,0.420,35,N
210,2,81,60,22,0,27.7,0.290,25,N
483,0,84,82,31,125,38.2,0.233,23,N
743,9,140,94,0,0,32.7,0.734,45,Y
408,8,197,74,0,0,25.9,1.191,39,Y
468,8,120,0,0,0,30.0,0.183,38,Y
283,7,161,86,0,0,30.4,0.165,47,Y
167,4,120,68,0,0,29.6,0.709,34,N


In [6]:
### Get training and testing array.
X_train = np.array(train_df[df_info.ohe_feature_names])
y_train = np.array(train_df[df_info.target_name])
X_test = np.array(test_df[df_info.ohe_feature_names])
y_test = np.array(test_df[df_info.target_name])

In [7]:
### Train models.
# models = train_three_models(X_train, y_train)

### Save models.
# save_three_models(models, dataset_name)

In [8]:
### Load models
models = load_three_models(X_train.shape[-1], dataset_name)

/Users/jrhs/miniforge3/envs/tf_mac/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


In [9]:
### Print out accuracy on testset.
evaluation_test(models, X_test, y_test)

DT: [0.7727] | RF [0.7987] | NN [0.7662]


# Alibi

## 1. Counterfactual Prototype

In [10]:
from utils.cf_proto import Recorder

In [11]:
### Setting up the CF generating amount.
num_instances = 3
num_cf_per_instance = 1

In [12]:
# Generate CF
results = generate_cf_proto_result(
    df_info,
    train_df,
    models,
    num_instances,
    num_cf_per_instance,
    X_train,
    X_test,
    y_test,
    max_iters=1000,
    models_to_run=['rfc', 'nn'],
    output_int=True
)
result_dfs = process_result(results, df_info)


Finding counterfactual for rfc
instance 0
CF 0
Found CF
instance 1
CF 0
CF not found
instance 2
CF 0
Found CF
Finding counterfactual for nn
instance 0
CF 0
Found CF
instance 1
CF 0
CF not found
instance 2
CF 0
CF not found


In [13]:
i = 0
example_input = X_test[i:i+1]

In [14]:
print(Recorder.wrapped_models['nn'].predict(example_input))
print(Recorder.wrapped_models['dt'].predict(example_input))
print(Recorder.wrapped_models['rfc'].predict(example_input))

[[1]]
[[1]]
[[1]]


In [15]:
print(Recorder.wrapped_models['nn'].predict_proba(example_input))
print(Recorder.wrapped_models['dt'].predict_proba(example_input))
print(Recorder.wrapped_models['rfc'].predict_proba(example_input))

[[0 1]]
[[0 1]]
[[0 1]]


In [21]:
# result_dfs['nn']['Found']

0    Y
0    N
0    N
Name: Found, dtype: object

In [ ]:
#  [Proto] learning_rate_init=0.01
#  [Watcher] learning_rate = 0.1

In [16]:
### Save result as file.
save_result_as_csv("proto", dataset_name, result_dfs)

Result has been saved to ./results/proto_diabetes
